In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset2/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset2/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset2/testX.npy')
y_test = np.load('gdrive/My Drive/dataset2/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from tensorflow import keras
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
# from keras.utils import to_categorical
from matplotlib import pyplot
from keras import regularizers
import tensorflow as tfs

In [ ]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 32
x_train = x_train.reshape((x_train.shape[0], n_steps, n_length, n_features))
x_test = x_test.reshape((x_test.shape[0], n_steps, n_length, n_features))

In [ ]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [ ]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, None, 28, 64)     1984      
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, None, 24, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, None, 20, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, None, 16, 64)     20544     
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, None, 16, 64)    

In [ ]:
# fit network
history = model2.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/200
278/278 [==============================] - 49s 166ms/step - loss: 1.0132 - accuracy: 0.6860 - val_loss: 0.2890 - val_accuracy: 0.9170
Epoch 2/200
278/278 [==============================] - 47s 168ms/step - loss: 0.2873 - accuracy: 0.9196 - val_loss: 0.1612 - val_accuracy: 0.9565
Epoch 3/200
278/278 [==============================] - 47s 167ms/step - loss: 0.1933 - accuracy: 0.9444 - val_loss: 0.1188 - val_accuracy: 0.9674
Epoch 4/200
278/278 [==============================] - 46s 165ms/step - loss: 0.1501 - accuracy: 0.9582 - val_loss: 0.0990 - val_accuracy: 0.9709
Epoch 5/200
278/278 [==============================] - 46s 166ms/step - loss: 0.1253 - accuracy: 0.9641 - val_loss: 0.1084 - val_accuracy: 0.9684
Epoch 6/200
278/278 [==============================] - 46s 166ms/step - loss: 0.1020 - accuracy: 0.9703 - val_loss: 0.0841 - val_accuracy: 0.9751
Epoch 7/200
278/278 [==============================] - 46s 166ms/step - loss: 0.0913 - accuracy: 0.9734 - val_loss: 0.0788 -

In [ ]:
# evaluate model
loss, accuracy = model2.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9899
Testing Accuracy: 0.9577
